# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
data = pd.read_csv('/content/drive/MyDrive/Data_sets/customer_product_sales.csv')

In [4]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68584 entries, 0 to 68583
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   CustomerID   68584 non-null  int64 
 1   FirstName    68584 non-null  object
 2   LastName     68584 non-null  object
 3   SalesID      68584 non-null  int64 
 4   ProductID    68584 non-null  int64 
 5   ProductName  68584 non-null  object
 6   Quantity     68584 non-null  int64 
dtypes: int64(4), object(3)
memory usage: 3.7+ MB


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [32]:
purchase_consumer= data.groupby(['CustomerID', 'ProductName']).agg({'Quantity':'sum'})
purchase_consumer

Quantity
CustomerID ProductName                            
33         Apricots - Dried                      1
           Assorted Desserts                     1
           Bandage - Flexible Neon               1
           Bar Mix - Pina Colada, 355 Ml         1
           Beans - Kidney, Canned                1
...                                            ...
98200      Vol Au Vents                         50
           Wasabi Powder                        25
           Wine - Fume Blanc Fetzer             25
           Wine - Hardys Bankside Shiraz        25
           Yogurt - French Vanilla              25

[63628 rows x 1 columns]

## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [39]:
Tabla= pd.pivot_table(df,
                      index= 'ProductName',
                 columns= 'CustomerID',
                 values= 'Quantity',
                 ).fillna(0)

In [11]:
Tabla.transpose()

ProductName,Anchovy Paste - 56 G Tube,"Appetizer - Mini Egg Roll, Shrimp",Appetizer - Mushroom Tart,Appetizer - Sausage Rolls,Apricots - Dried,Apricots - Halves,Apricots Fresh,Arizona - Green Tea,Artichokes - Jerusalem,Assorted Desserts,...,"Wine - White, Colubia Cresh","Wine - White, Mosel Gold","Wine - White, Schroder And Schyl",Wine - Wyndham Estate Bin 777,Wonton Wrappers,Yeast Dry - Fermipan,Yoghurt Tubes,"Yogurt - Blueberry, 175 Gr",Yogurt - French Vanilla,Zucchini - Yellow
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
200,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
264,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
356,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
412,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97928,0.0,0.0,0.0,25.0,0.0,50.0,0.0,25.0,0.0,0.0,...,0.0,25.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
98069,0.0,0.0,0.0,25.0,0.0,25.0,0.0,0.0,0.0,25.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
98159,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,50.0,0.0,0.0,0.0,0.0,25.0,0.0,0.0,0.0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [14]:
euclidian = 1/(1+squareform(pdist(Tabla, metric='euclidean'))) #generacion de la distancia

distance = pd.DataFrame(euclidian, index= Tabla.index, columns= Tabla.index) #creacion de la matriz de similitud

distance.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.077421,0.087047,0.081800,0.080634,0.082709,0.074573,0.083020,0.081503,0.080070,...,0.004811,0.004669,0.004412,0.005019,0.004312,0.004515,0.004583,0.004355,0.004167,0.004333
200,0.077421,1.000000,0.078448,0.076435,0.073693,0.075255,0.075956,0.076435,0.077674,0.076923,...,0.004824,0.004681,0.004431,0.005047,0.004311,0.004521,0.004614,0.004367,0.004166,0.004335
264,0.087047,0.078448,1.000000,0.080070,0.081800,0.080350,0.076923,0.080634,0.082100,0.078448,...,0.004822,0.004674,0.004416,0.005035,0.004322,0.004543,0.004595,0.004365,0.004179,0.004333
356,0.081800,0.076435,0.080070,1.000000,0.076435,0.078187,0.075025,0.082403,0.077171,0.075956,...,0.004816,0.004671,0.004416,0.005038,0.004310,0.004526,0.004578,0.004365,0.004175,0.004339
412,0.080634,0.073693,0.081800,0.076435,1.000000,0.078711,0.075025,0.082403,0.078187,0.078448,...,0.004810,0.004702,0.004414,0.005034,0.004318,0.004530,0.004578,0.004367,0.004177,0.004349


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [26]:
custom200 = pd.DataFrame(distance[200].sort_values(ascending=False)[1:].head(5))
custom200 = custom200.index
custom200

Int64Index([3317, 1072, 3535, 1920, 3909], dtype='int64', name='CustomerID')

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [43]:
#data created in step 1 = df
#data created n step 2 = Tabla

similar = pd.DataFrame(Tabla, columns= custom200)
similar

CustomerID,3317,1072,3535,1920,3909
ProductName,,,,,
Anchovy Paste - 56 G Tube,0.0,0.0,1.0,0.0,0.0
"Appetizer - Mini Egg Roll, Shrimp",0.0,1.0,0.0,0.0,0.0
Appetizer - Mushroom Tart,0.0,0.0,0.0,0.0,0.0
Appetizer - Sausage Rolls,0.0,1.0,0.0,0.0,0.0
Apricots - Dried,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...
Yeast Dry - Fermipan,0.0,1.0,0.0,0.0,0.0
Yoghurt Tubes,0.0,0.0,1.0,0.0,0.0
"Yogurt - Blueberry, 175 Gr",0.0,0.0,0.0,0.0,0.0


In [47]:
similarities200 = purchase_consumer.loc[custom200]
similarities200

Quantity
CustomerID ProductName                             
3317       Bay Leaf                               1
           Beef - Ground Medium                   1
           Beef - Rib Eye Aaa                     1
           Butter - Unsalted                      1
           Cheese - Victor Et Berthold            1
...                                             ...
3909       Wine - Charddonnay Errazuriz           2
           Wine - Fume Blanc Fetzer               1
           Wine - Magnotta, Merlot Sr Vqa         1
           Wine - Toasted Head                    1
           Wine - Valpolicella Masi               1

[262 rows x 1 columns]

## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [48]:
total_5_similar_200 = similarities200.groupby('ProductName').agg({'Quantity': sum}).sort_values('Quantity', ascending=False)
total_5_similar_200

,Quantity
ProductName,
Soup - Campbells Bean Medley,4
Muffin - Carrot Individual Wrap,3
Bay Leaf,3
Pork - Kidney,3
"Pepper - Black, Whole",3
...,...
Kiwi,1
Knife Plastic - White,1
Lambcasing,1


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [50]:
filter = pd.merge(left= total_5_similar_200, right=Tabla[200], on='ProductName' )
filter[filter[200]==0].head(5)


,Quantity,200
ProductName,,
Soup - Campbells Bean Medley,4,0.0
Muffin - Carrot Individual Wrap,3,0.0
Bay Leaf,3,0.0
Pork - Kidney,3,0.0
Wanton Wrap,3,0.0


## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [53]:
recomm_dict = {}
recomend = {}

def recommendations(customer_id):
  euclidian = 1/(1+squareform(pdist(Tabla.transpose(), metric='euclidean'))) #step 3
  distance = pd.DataFrame(euclidian, index= Tabla.T.index, columns= Tabla.T.index) #step 3
  similarities = list(distance[customer_id].sort_values(ascending=False)[1:6].index) #step 4
  simil_custom_id = purchase_consumer.loc[similarities] #step 5
  total_5_similar_custom_id = simil_custom_id.groupby('ProductName').sum().sort_values('Quantity', ascending=False) #step6
  Merge = total_5_similar_custom_id.merge(Tabla, on='ProductName') #step 7
  recommendation = list(Merge[Merge[customer_id] == 0].sort_values('Quantity', ascending=False).head(5).index) #step 7
  recomm_dict[customer_id] = recommendation 
  return recommendation



In [54]:
for i in data['CustomerID'].unique():
    recommendations(i)

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [55]:
pd.DataFrame.from_dict(recomm_dict).T



,0,1,2,3,4
61288,Jagermeister,Chicken - Soup Base,Lime Cordial - Roses,Macaroons - Two Bite Choc,Flavouring - Orange
77352,Wine - Valpolicella Masi,Tahini Paste,Tuna - Salad Premix,Chicken - Soup Base,Isomalt
40094,Bread - Italian Corn Meal Poly,Puree - Mocha,Tuna - Salad Premix,Sherry - Dry,Beer - Sleemans Cream Ale
23548,Squid - Tubes / Tenticles 10/20,Wanton Wrap,Puree - Mocha,Pernod,Sprouts - Baby Pea Tendrils
78981,Vanilla Beans,Lettuce - Frisee,"Yogurt - Blueberry, 175 Gr",Cinnamon Buns Sticky,Pop Shoppe Cream Soda
...,...,...,...,...,...
49005,"Pasta - Penne, Rigate, Dry",Browning Caramel Glace,Tea - Decaf Lipton,"Garlic - Primerba, Paste","Wine - Red, Harrow Estates, Cab"
41286,Veal - Inside,Wonton Wrappers,"Veal - Inside, Choice",Bread Crumbs - Japanese Style,Wasabi Powder
85878,Olives - Kalamata,Wine - White Cab Sauv.on,Veal - Eye Of Round,Bread - Calabrese Baguette,Cheese - Wine
68506,"Veal - Brisket, Provimi,bnls",Beef - Ground Medium,"Rosemary - Primerba, Paste",Scallops - 10/20,"Oregano - Dry, Rubbed"


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [59]:
recomm_dict_2={}
recomm_2 = {}

def recommendations_2(customer_id):
  citiblock = 1/(1+squareform(pdist(Tabla.transpose(), metric='cityblock'))) #step 3
  distance = pd.DataFrame(citiblock, index= Tabla.T.index, columns= Tabla.T.index) #step 3
  similarities = list(distance[customer_id].sort_values(ascending=False)[1:6].index) #step 4
  simil_custom_id = purchase_consumer.loc[similarities] #step 5
  total_5_similar_custom_id = simil_custom_id.groupby('ProductName').sum().sort_values('Quantity', ascending=False) #step6
  Merge = total_5_similar_custom_id.merge(Tabla, on='ProductName')
  recommendation = list(Merge[Merge[customer_id] == 0].sort_values('Quantity', ascending=False).head(5).index)
  recomm_dict_2[customer_id] = recommendation 
  return recommendation

In [60]:
for i in data['CustomerID'].unique():
    recommendations_2(i)

In [62]:
pd.DataFrame.from_dict(recomm_dict_2).T


,0,1,2,3,4
61288,Soup - Campbells Bean Medley,Sausage - Liver,Butter - Unsalted,Bay Leaf,"Pepper - Black, Whole"
77352,Tia Maria,"Sole - Dover, Whole, Fresh",Veal - Sweetbread,Foam Dinner Plate,Juice - Lime
40094,Knife Plastic - White,Wine - Ej Gallo Sierra Valley,Beef Wellington,Soup - Campbells Bean Medley,Butter - Unsalted
23548,Beef - Texas Style Burger,Squid - Tubes / Tenticles 10/20,"Cheese - Boursin, Garlic / Herbs",Wine - Chardonnay South,Ice Cream Bar - Hageen Daz To
78981,Butter - Unsalted,"Veal - Inside, Choice",Veal - Sweetbread,Beef - Montreal Smoked Brisket,Tia Maria
...,...,...,...,...,...
49005,Curry Paste - Madras,Crush - Cream Soda,Muffin - Carrot Individual Wrap,Wine - Ej Gallo Sierra Valley,"Cheese - Boursin, Garlic / Herbs"
41286,Pork - Kidney,Butter - Unsalted,Knife Plastic - White,Muffin - Carrot Individual Wrap,Wine - Blue Nun Qualitatswein
85878,Sobe - Tropical Energy,"Spoon - Soup, Plastic",Ice Cream Bar - Hageen Daz To,Tomatoes Tear Drop,Muffin - Carrot Individual Wrap
68506,"Pepper - Black, Whole",Halibut - Steaks,Cod - Black Whole Fillet,"Pepper - Paprika, Hungarian",Pork - Hock And Feet Attached
